In [ ]:
import requests

# Your Tavily API Key
api_key = "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"
url = "https://api.tavily.com/search"  # Ensure this is the correct endpoint

# Define headers
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"  # Ensure JSON request format
}

# Define request payload
payload = {
    "query": "Latest AI breakthroughs 2025",
    "num_results": 3  # Adjust based on API limits
}

# Make POST request
response = requests.post(url, json=payload, headers=headers)

# Print debug information
print("Status Code:", response.status_code)
print("Response Text:", response.text)


Status Code: 200
Response Text: {"query":"Latest AI breakthroughs 2025","follow_up_questions":null,"answer":null,"images":[],"results":[{"title":"Latest AI Breakthroughs and News - February 2025 | News","url":"https://www.crescendo.ai/news/latest-ai-news-and-updates","content":"Summary: OpenAI has announced GPT-4.5, its largest and most advanced AI model to date. Summary: Interview Kickstart has introduced a new Generative AI (GenAI) course designed for professionals aiming to enhance their skills in artificial intelligence. Baidu to Open-Source Latest Ernie AI Model Summary: Elon Musk’s social media platform, X, has announced an upgrade to its AI chatbot, Grok, introducing advanced image editing features powered by the Aurora model. Summary: Chinese startup DeepSeek unveiled n AI model that rivals leading U.S. technologies but operates with significantly fewer resources. Summary: Google announced significant AI advancements, including the release of Gemini 2.0, their most capable mode

In [ ]:
import requests

# Tavily API Key
api_key = "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"
url = "https://api.tavily.com/search"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

payload = {
    "query": "Latest AI breakthroughs 2025",
    "num_results": 5
}

# Fetch search results
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    results = data.get("results", [])  # Extract search results

    for idx, result in enumerate(results, 1):
        title = result.get("title", "No Title")
        url = result.get("url", "No URL")
        content = result.get("content", "No Summary Available")

        print(f"{idx}. {title}")
        print(f"   URL: {url}")
        print(f"   Summary: {content}\n")
else:
    print("Error fetching data:", response.text)


1. Latest AI Breakthroughs and News - February 2025 | News
   URL: https://www.crescendo.ai/news/latest-ai-news-and-updates
   Summary: Summary: OpenAI has announced GPT-4.5, its largest and most advanced AI model to date. Summary: Interview Kickstart has introduced a new Generative AI (GenAI) course designed for professionals aiming to enhance their skills in artificial intelligence. Baidu to Open-Source Latest Ernie AI Model Summary: Elon Musk’s social media platform, X, has announced an upgrade to its AI chatbot, Grok, introducing advanced image editing features powered by the Aurora model. Summary: Chinese startup DeepSeek unveiled n AI model that rivals leading U.S. technologies but operates with significantly fewer resources. Summary: Google announced significant AI advancements, including the release of Gemini 2.0, their most capable model to date, featuring agentic capabilities for developers, enterprises, and individuals. Source: Google AI Blog Source: Google AI Blog

2. Top 1